In [2]:
import re
import cv2
import numpy as np
import gradio as gr

blocks = gr.Blocks()

# replace multiple spaces with single space
# Sometimes you want to copy command from other terminal result and paste it to your terminal but \n or multiple spaces causes problem...                         
def replace_multiple_spaces_with_single_space(string:str) -> str:
    return (re.sub(r'\s+', ' ', string, flags=re.I)) 

import PIL
from PIL import Image
import numpy as np

In [3]:
# concat images horizontally, useful for combining poses.

def concat_images_horizontally(images):
    widths, heights = zip(*(i.size for i in images))
    total_width = sum(widths)
    max_height = max(heights)
    new_im = Image.new('RGB', (total_width, max_height))
    x_offset = 0
    for im in images:
      new_im.paste(im, (x_offset,0))
      x_offset += im.size[0]
    return new_im

def concat_from_files_horizontally(files):
    images = []
    for file in files:
        images.append(Image.open(file))
    return concat_images_horizontally(images)

import glob


def concat_from_glob_horizontally(glob_pattern:str) -> Image.Image:
    """
    Concat images horizontally from glob pattern (directory path)
    ex) concat_from_glob_horizontally("/home/user/*.png")
    """
    files = glob.glob(glob_pattern)
    return concat_from_files_horizontally(files)

# using base64 to decode string

In [12]:
import base64

def decode_base64_to_string(base64_string):
    return base64.b64decode(base64_string).decode('utf-8')

#def autogen_mask_blur(pose_image, radius = 5, threshold = 40):
    # convert to grayscale. Then if value is less than 40, set to 0, otherwise set to 255
    # Then for each white pixel, increase brightness for circle with radius to new image
    # Then apply gaussian blur to new image
    # apply threshold again
    # apply connected components to get separate objects
    # apply different color for each object


In [13]:
mask_image = "C:\RegionalMaskFromOpenPose/2.png"
mask_image2 = cv2.imread(mask_image, cv2.COLOR_BGR2GRAY)
#cv2.imshow('test',mask_image2)
#cv2.waitKey(0)
#cv2.destoryAllWinodws()
cv2.imwrite('./2mask_image2.png',mask_image2)

# mask_image2 to make inverted
mask_image3 = cv2.bitwise_not(mask_image2)
cv2.imwrite('./3mask_image3.png',mask_image3)

True

In [14]:
# convert to grayscale
gray = cv2.cvtColor(mask_image3, cv2.COLOR_BGR2GRAY)
cv2.imwrite('./3gray.png',gray)
gray

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [31]:
threshold =60
gaussian = 9 # 홀수 여야함
radius = 50
# threshold to get a mask
_, mask = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
cv2.imwrite('./4mask.png', mask)

# create a new image with the same size as the input image
new_image = np.zeros_like(gray)
cv2.imwrite('./5new_image.png', new_image)
# for each white pixel in the mask, increase brightness for circle with radius in new image
indices = np.where(mask == 255)
for z in zip(*indices):
    y, x = z
    cv2.circle(new_image, (x, y), radius, (255, 255, 255), -1)

# apply gaussian blur to new image
blurred = cv2.GaussianBlur(new_image, (gaussian, gaussian), 0)
# if debug_image_output is not None:
#    debug_image_output.value = blurred
cv2.imwrite('./6blurred.png', blurred)

True

In [35]:
# fill the inside of the mask of blurred
_, mask = cv2.threshold(blurred, 40, 255, cv2.THRESH_BINARY)

# apply connected components to get separate objects
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)
# apply different color for each object
output = np.zeros_like(mask)
for i in range(1, num_labels):
    mask = np.where(labels == i, 255, 0)
    output = cv2.add(output, mask, dtype=cv2.CV_32F)
cv2.imwrite('./7output.png', output)



True

In [36]:
nelem = 0
height, width = blurred.shape
for x in range(height):
    for y in range(width):
        if blurred[x, y] == 0:
            nelem += 1
            cv2.floodFill(blurred, None, (y, x), nelem)
           #  cv2.floodFill(blurred, None, (y, x), (nelem, nelem, nelem))
cv2.imwrite('./8blurred.png', blurred)  


True

In [16]:
# threshold to get a mask
_, mask = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY)
cv2.imwrite('./7mask.png', mask)

# assert mask is 1 channel
assert mask.shape == (mask_image3.shape[0], mask_image3.shape[1]), "mask should have 1 channel"

# separate objects with connected components
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)
print(num_labels, labels, stats, centroids)

# apply different color for each object
# now use 3 channels
empty = np.zeros_like(mask_image3)
# randomly generate colors
colors = np.random.randint(0, 255, size=(num_labels, 3), dtype=np.uint8)
# set background to white
colors[0] = (255, 255, 255)
# for each object, set color to the object
for i in range(1, num_labels):
    empty[labels == i] = colors[i]

Image.fromarray(empty), Image.fromarray(new_image), blurred


3 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [[      0       0    1886    2649 4694948]
 [      0     940    1661    1709  295949]
 [    600    1499     181      69    5117]] [[ 964.34536293 1289.07757956]
 [ 600.34512365 1874.43951154]
 [ 687.96619113 1530.55169044]]


(<PIL.Image.Image image mode=RGB size=1886x2649>,
 <PIL.Image.Image image mode=L size=1886x2649>,
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8))